# CrewAI Short-Term Memory with Couchbase GSI Vector Search

## Overview

This tutorial shows how to implement a custom memory backend for CrewAI agents using Couchbase's high-performance GSI (Global Secondary Index) vector search. CrewAI agents can retain and recall information across interactions, making them more contextually aware and effective. We'll demonstrate measurable performance improvements with GSI optimization.

**Key Features:**
- Custom CrewAI memory storage with Couchbase GSI vector search
- High-performance semantic memory retrieval
- Agent memory persistence across conversations
- Performance benchmarks showing GSI benefits

**Requirements:** Couchbase Server 8.0+ or Capella with Query Service enabled.

## Prerequisites

### Couchbase Setup

1. **Create Capella Account:** Deploy a [free tier cluster](https://cloud.couchbase.com/sign-up)
2. **Enable Query Service:** Required for GSI vector search
3. **Configure Access:** Set up database credentials and network security
4. **Create Bucket:** Manual bucket creation recommended for Capella

## Understanding Agent Memory

### Why Memory Matters for AI Agents

Memory in AI agents is a crucial capability that allows them to retain and utilize information across interactions, making them more effective and contextually aware. Without memory, agents would be limited to processing only the immediate input, lacking the ability to build upon past experiences or maintain continuity in conversations.

#### Types of Memory in AI Agents

**Short-term Memory:**
- Retains recent interactions and context
- Typically spans the current conversation or session  
- Helps maintain coherence within a single interaction flow
- In CrewAI, this is what we're implementing with the Couchbase storage

**Long-term Memory:**
- Stores persistent knowledge across multiple sessions
- Enables agents to recall past interactions even after long periods
- Helps build cumulative knowledge about users, preferences, and past decisions
- While this implementation is labeled as "short-term memory", the Couchbase storage backend can be effectively used for long-term memory as well, thanks to Couchbase's persistent storage capabilities and enterprise-grade durability features

#### How Memory Works in Agents

Memory in AI agents typically involves:
- **Storage**: Information is encoded and stored in a database (like Couchbase, ChromaDB, or other vector stores)
- **Retrieval**: Relevant memories are fetched based on semantic similarity to current context
- **Integration**: Retrieved memories are incorporated into the agent's reasoning process

The vector-based approach (using embeddings) is particularly powerful because it allows for semantic search - finding memories that are conceptually related to the current context, not just exact keyword matches.

#### Benefits of Memory in AI Agents

- **Contextual Understanding**: Agents can refer to previous parts of a conversation
- **Personalization**: Remembering user preferences and past interactions
- **Learning and Adaptation**: Building knowledge over time to improve responses
- **Task Continuity**: Resuming complex tasks across multiple interactions
- **Collaboration**: In multi-agent systems like CrewAI, memory enables agents to build on each other's work

#### Memory in CrewAI Specifically

In CrewAI, memory serves several important functions:
- **Agent Specialization**: Each agent can maintain its own memory relevant to its expertise
- **Knowledge Transfer**: Agents can share insights through memory when collaborating on tasks
- **Process Continuity**: In sequential processes, later agents can access the work of earlier agents
- **Contextual Awareness**: Agents can reference previous findings when making decisions

## Setup and Installation

### Install Required Libraries

Install the necessary packages for CrewAI, Couchbase integration, and OpenAI embeddings.

In [1]:
%pip install --quiet crewai==0.186.1 langchain-couchbase==0.5.0rc1 langchain-openai==0.3.33 python-dotenv==1.1.1

Note: you may need to restart the kernel to use updated packages.


### Import Required Modules

Import libraries for CrewAI memory storage, Couchbase GSI vector search, and OpenAI embeddings.

In [2]:
from typing import Any, Dict, List, Optional
import os
import logging
from datetime import timedelta
from dotenv import load_dotenv
from crewai.memory.storage.rag_storage import RAGStorage
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai import Agent, Crew, Task, Process
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.diagnostics import PingState, ServiceType
from langchain_couchbase.vectorstores import CouchbaseQueryVectorStore
from langchain_couchbase.vectorstores import DistanceStrategy
from langchain_couchbase.vectorstores import IndexType
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import time
import json
import uuid

# Configure logging (disabled)
logging.basicConfig(level=logging.CRITICAL)
logger = logging.getLogger(__name__)

### Environment Configuration

Configure environment variables for secure access to Couchbase and OpenAI services. Create a `.env` file with your credentials.

In [3]:
load_dotenv("./.env")

# Verify environment variables
required_vars = ['OPENAI_API_KEY', 'CB_HOST', 'CB_USERNAME', 'CB_PASSWORD']
for var in required_vars:
    if not os.getenv(var):
        raise ValueError(f"{var} environment variable is required")

## Understanding GSI Vector Search

### GSI Vector Index Types

Couchbase offers two types of GSI vector indexes for different use cases:

**Hyperscale Vector Indexes (BHIVE):**
- Best for pure vector searches - content discovery, recommendations, semantic search
- High performance with low memory footprint - designed to scale to billions of vectors
- Optimized for concurrent operations - supports simultaneous searches and inserts
- Use when: You primarily perform vector-only queries without complex scalar filtering
- Ideal for: Large-scale semantic search, recommendation systems, content discovery

**Composite Vector Indexes:**
- Best for filtered vector searches - combines vector search with scalar value filtering
- Efficient pre-filtering - scalar attributes reduce the vector comparison scope
- Use when: Your queries combine vector similarity with scalar filters that eliminate large portions of data
- Ideal for: Compliance-based filtering, user-specific searches, time-bounded queries

For this CrewAI memory implementation, we'll use **BHIVE** as it's optimized for pure semantic search scenarios typical in AI agent memory systems.

### Understanding Index Configuration

The `index_description` parameter controls how Couchbase optimizes vector storage and search performance through centroids and quantization:

**Format**: `'IVF[<centroids>],{PQ|SQ}<settings>'`

**Centroids (IVF - Inverted File):**
- Controls how the dataset is subdivided for faster searches
- More centroids = faster search, slower training  
- Fewer centroids = slower search, faster training
- If omitted (like IVF,SQ8), Couchbase auto-selects based on dataset size

**Quantization Options:**
- SQ (Scalar Quantization): SQ4, SQ6, SQ8 (4, 6, or 8 bits per dimension)
- PQ (Product Quantization): PQ<subquantizers>x<bits> (e.g., PQ32x8)
- Higher values = better accuracy, larger index size

**Common Examples:**
- IVF,SQ8 - Auto centroids, 8-bit scalar quantization (good default)
- IVF1000,SQ6 - 1000 centroids, 6-bit scalar quantization  
- IVF,PQ32x8 - Auto centroids, 32 subquantizers with 8 bits

For detailed configuration options, see the [Quantization & Centroid Settings](https://preview.docs-test.couchbase.com/docs-server-DOC-12565_vector_search_concepts/server/current/vector-index/hyperscale-vector-index.html#algo_settings).

For more information on GSI vector indexes, see [Couchbase GSI Vector Documentation](https://docs.couchbase.com/server/current/vector-index/use-vector-indexes.html).


## Custom CouchbaseStorage Implementation

### CouchbaseStorage Class

This class extends CrewAI's `RAGStorage` to provide GSI vector search capabilities for agent memory.

In [4]:
class CouchbaseStorage(RAGStorage):
    """
    Extends RAGStorage to handle embeddings for memory entries using Couchbase GSI Vector Search.
    """

    def __init__(self, type: str, allow_reset: bool = True, embedder_config: Optional[Dict[str, Any]] = None, crew: Optional[Any] = None):
        """Initialize CouchbaseStorage with GSI vector search configuration."""
        super().__init__(type, allow_reset, embedder_config, crew)
        self._initialize_app()

    def search(
        self,
        query: str,
        limit: int = 3,
        filter: Optional[dict] = None,
        score_threshold: float = 0,
    ) -> List[Dict[str, Any]]:
        """
        Search memory entries using GSI vector similarity.
        """
        try:
            # Add type filter
            search_filter = {"memory_type": self.type}
            if filter:
                search_filter.update(filter)

            # Execute search using GSI vector search
            results = self.vector_store.similarity_search_with_score(
                query,
                k=limit,
                filter=search_filter
            )
            
            # Format results and deduplicate by content
            seen_contents = set()
            formatted_results = []
            
            for i, (doc, distance) in enumerate(results):
                # Note: In GSI vector search, lower distance indicates higher similarity
                if distance <= (1.0 - score_threshold):  # Convert threshold for GSI distance metric
                    content = doc.page_content
                    if content not in seen_contents:
                        seen_contents.add(content)
                        formatted_results.append({
                            "id": doc.metadata.get("memory_id", str(i)),
                            "metadata": doc.metadata,
                            "context": content,
                            "distance": float(distance)  # Changed from score to distance
                        })
            
            logger.info(f"Found {len(formatted_results)} unique results for query: {query}")
            return formatted_results

        except Exception as e:
            logger.error(f"Search failed: {str(e)}")
            return []

    def save(self, value: Any, metadata: Dict[str, Any]) -> None:
        """
        Save a memory entry with metadata.
        """
        try:
            # Generate unique ID
            memory_id = str(uuid.uuid4())
            timestamp = int(time.time() * 1000)
            
            # Prepare metadata (create a copy to avoid modifying references)
            if not metadata:
                metadata = {}
            else:
                metadata = metadata.copy()  # Create a copy to avoid modifying references
                
            # Process agent-specific information if present
            agent_name = metadata.get('agent', 'unknown')
                
            # Clean up value if it has the typical LLM response format
            value_str = str(value)
            if "Final Answer:" in value_str:
                # Extract just the actual content - everything after "Final Answer:"
                parts = value_str.split("Final Answer:", 1)
                if len(parts) > 1:
                    value = parts[1].strip()
                    logger.info(f"Cleaned up response format for agent: {agent_name}")
            elif value_str.startswith("Thought:"):
                # Handle thought/final answer format
                if "Final Answer:" in value_str:
                    parts = value_str.split("Final Answer:", 1)
                    if len(parts) > 1:
                        value = parts[1].strip()
                        logger.info(f"Cleaned up thought process format for agent: {agent_name}")
            
            # Update metadata
            metadata.update({
                "memory_id": memory_id,
                "memory_type": self.type,
                "timestamp": timestamp,
                "source": "crewai"
            })

            # Log memory information for debugging
            value_preview = str(value)[:100] + "..." if len(str(value)) > 100 else str(value)
            metadata_preview = {k: v for k, v in metadata.items() if k != "embedding"}
            logger.info(f"Saving memory for Agent: {agent_name}")
            logger.info(f"Memory value preview: {value_preview}")
            logger.info(f"Memory metadata: {metadata_preview}")
            
            # Convert value to string if needed
            if isinstance(value, (dict, list)):
                value = json.dumps(value)
            elif not isinstance(value, str):
                value = str(value)

            # Save to GSI vector store
            self.vector_store.add_texts(
                texts=[value],
                metadatas=[metadata],
                ids=[memory_id]
            )
            logger.info(f"Saved memory {memory_id}: {value[:100]}...")

        except Exception as e:
            logger.error(f"Save failed: {str(e)}")
            raise

    def reset(self) -> None:
        """Reset the memory storage if allowed."""
        if not self.allow_reset:
            return

        try:
            # Delete documents of this memory type
            self.cluster.query(
                f"DELETE FROM `{self.bucket_name}`.`{self.scope_name}`.`{self.collection_name}` WHERE memory_type = $type",
                type=self.type
            ).execute()
            logger.info(f"Reset memory type: {self.type}")
        except Exception as e:
            logger.error(f"Reset failed: {str(e)}")
            raise

    def _initialize_app(self):
        """Initialize Couchbase connection and GSI vector store."""
        try:
            # Initialize embeddings
            if self.embedder_config and self.embedder_config.get("provider") == "openai":
                self.embeddings = OpenAIEmbeddings(
                    openai_api_key=os.getenv('OPENAI_API_KEY'),
                    model=self.embedder_config.get("config", {}).get("model", "text-embedding-3-small")
                )
            else:
                self.embeddings = OpenAIEmbeddings(
                    openai_api_key=os.getenv('OPENAI_API_KEY'),
                    model="text-embedding-3-small"
                )

            # Connect to Couchbase
            auth = PasswordAuthenticator(
                os.getenv('CB_USERNAME', ''),
                os.getenv('CB_PASSWORD', '')
            )
            options = ClusterOptions(auth)
            
            # Initialize cluster connection
            self.cluster = Cluster(os.getenv('CB_HOST', ''), options)
            self.cluster.wait_until_ready(timedelta(seconds=5))

            # Check Query service (required for GSI vector search)
            ping_result = self.cluster.ping()
            query_available = False
            for service_type, endpoints in ping_result.endpoints.items():
                if service_type.name == 'Query':  # Query Service for GSI
                    for endpoint in endpoints:
                        if endpoint.state == PingState.OK:
                            query_available = True
                            logger.info(f"Query service is responding at: {endpoint.remote}")
                            break
                    break
            if not query_available:
                raise RuntimeError("Query service not found or not responding. GSI vector search requires Query Service.")
            
            # Set up storage configuration
            self.bucket_name = os.getenv('CB_BUCKET_NAME', 'vector-search-testing')
            self.scope_name = os.getenv('SCOPE_NAME', 'shared')
            self.collection_name = os.getenv('COLLECTION_NAME', 'crew')
            self.index_name = os.getenv('INDEX_NAME', 'vector_search_crew_gsi')

            # Initialize GSI vector store
            self.vector_store = CouchbaseQueryVectorStore(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=self.scope_name,
                collection_name=self.collection_name,
                embedding=self.embeddings,
                distance_metric=DistanceStrategy.COSINE,
            )
            logger.info(f"Initialized CouchbaseStorage with GSI vector search for type: {self.type}")

        except Exception as e:
            logger.error(f"Initialization failed: {str(e)}")
            raise

## Memory Performance Testing

### Initialize Storage and Test Basic Operations

First, let's test memory storage and retrieval without GSI optimization to establish a baseline.

In [6]:
# Initialize storage
storage = CouchbaseStorage(
    type="short_term",
    embedder_config={
        "provider": "openai",
        "config": {"model": "text-embedding-3-small"}
    }
)

# Reset storage
storage.reset()

# Test storage
test_memory = "Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased.'"
test_metadata = {"category": "sports", "test": "initial_memory"}
storage.save(test_memory, test_metadata)

# Test baseline search performance without GSI index
import time

def test_memory_search_performance(storage, query, label="Memory Search"):
    """Test memory search performance and return timing metrics"""
    print(f"\n[{label}] Testing query: '{query}'")
    start_time = time.time()
    
    try:
        results = storage.search(query, limit=3)
        end_time = time.time()
        search_time = end_time - start_time
        
        print(f"[{label}] Search completed in {search_time:.4f} seconds")
        print(f"[{label}] Found {len(results)} memories")
        
        if results:
            print(f"[{label}] Top result distance: {results[0]['distance']:.6f} (lower = more similar)")
            print(f"[{label}] Top result: {results[0]['context'][:100]}...")
        
        return search_time
    except Exception as e:
        print(f"[{label}] Search failed: {str(e)}")
        return None

# Test baseline performance without GSI index
test_query = "What did Guardiola say about Manchester City?"
baseline_time = test_memory_search_performance(storage, test_query, "Without GSI Index")
print(f"\nBaseline memory search time (without GSI): {baseline_time:.4f} seconds\n")


[Without GSI Index] Testing query: 'What did Guardiola say about Manchester City?'
[Without GSI Index] Search completed in 0.6425 seconds
[Without GSI Index] Found 1 memories
[Without GSI Index] Top result distance: 0.340161 (lower = more similar)
[Without GSI Index] Top result: Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a ...

Baseline memory search time (without GSI): 0.6425 seconds



### Create BHIVE GSI Index for Performance

Now create the high-performance BHIVE GSI index to optimize memory search speed.


In [7]:
# Create GSI BHIVE vector index for optimal performance
try:
    storage.vector_store.create_index(
        index_type=IndexType.BHIVE,
        # index_type=IndexType.COMPOSITE,  # Uncomment this line to create a COMPOSITE index instead
        index_name=storage.index_name,
        index_description="IVF,SQ8"  # Auto-selected centroids with 8-bit scalar quantization
    )
    print(f"Created GSI BHIVE vector index: {storage.index_name}")
    
    # Wait for index to become available
    print("Waiting for index to become available...")
    time.sleep(5)
    
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"GSI vector index '{storage.index_name}' already exists, continuing...")
    else:
        print(f"Error creating GSI index: {str(e)}")

Created GSI BHIVE vector index: vector_search_crew
Waiting for index to become available...


### Performance Comparison: Before vs After GSI Index

Test the same memory search with the BHIVE GSI index and compare performance.

In [9]:
# Test performance with GSI index
print("Testing memory search performance with BHIVE GSI optimization...")
gsi_time = test_memory_search_performance(storage, test_query, "With BHIVE GSI")

# Calculate performance improvement
if baseline_time and gsi_time:
    speedup = baseline_time / gsi_time if gsi_time > 0 else float('inf')
    time_saved = baseline_time - gsi_time
    percent_improvement = (time_saved / baseline_time) * 100
    
    print("\n" + "="*60)
    print("MEMORY SEARCH PERFORMANCE COMPARISON")
    print("="*60)
    print(f"Memory search without GSI: {baseline_time:.4f} seconds")
    print(f"Memory search with GSI:    {gsi_time:.4f} seconds")
    print(f"Performance improvement:    {speedup:.2f}x faster")
    print(f"Time saved:                {time_saved:.4f} seconds ({percent_improvement:.1f}% improvement)")
    
    print("\n" + "-"*60)
    print("IMPACT FOR AGENT MEMORY:")
    print("-"*60)
    print(f"• Faster memory retrieval improves agent response time")
    print(f"• {percent_improvement:.0f}% reduction in memory lookup time")
    print(f"• Better scalability for agents with large memory stores")
    print(f"• Enhanced real-time conversational performance")
else:
    print("\nUnable to calculate performance comparison due to search errors")

Testing memory search performance with BHIVE GSI optimization...

[With BHIVE GSI] Testing query: 'What did Guardiola say about Manchester City?'
[With BHIVE GSI] Search completed in 0.6698 seconds
[With BHIVE GSI] Found 1 memories
[With BHIVE GSI] Top result distance: 0.340165 (lower = more similar)
[With BHIVE GSI] Top result: Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a ...

MEMORY SEARCH PERFORMANCE COMPARISON
Memory search without GSI: 0.6425 seconds
Memory search with GSI:    0.6698 seconds
Performance improvement:    0.96x faster
Time saved:                -0.0273 seconds (-4.3% improvement)

------------------------------------------------------------
IMPACT FOR AGENT MEMORY:
------------------------------------------------------------
• Faster memory retrieval improves agent response time
• -4% reduction in memory lookup time
• Better scalability for agents with large memory stores
• Enhanced real-time conversational perfo

**Note on BHIVE GSI Performance:** The BHIVE GSI index may show slower performance for very small datasets (few documents) due to the additional overhead of maintaining the index structure. However, as the dataset scales up, the BHIVE GSI index becomes significantly faster than traditional vector searches. The initial overhead investment pays off dramatically with larger memory stores, making it essential for production agent deployments with substantial conversational history.

## CrewAI Agent Memory Integration

### Create Agents with Optimized Memory

Set up CrewAI agents that use our GSI-optimized Couchbase memory storage for fast, contextual memory retrieval.

In [10]:
# Initialize ShortTermMemory with our storage
memory = ShortTermMemory(storage=storage)

# Initialize language model
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7
)

# Create agents with memory
sports_analyst = Agent(
    role='Sports Analyst',
    goal='Analyze Manchester City performance',
    backstory='Expert at analyzing football teams and providing insights on their performance',
    llm=llm,
    memory=True,
    memory_storage=memory
)

journalist = Agent(
    role='Sports Journalist',
    goal='Create engaging football articles',
    backstory='Experienced sports journalist who specializes in Premier League coverage',
    llm=llm,
    memory=True,
    memory_storage=memory
)

# Create tasks
analysis_task = Task(
    description='Analyze Manchester City\'s recent performance based on Pep Guardiola\'s comments: "The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased."',
    agent=sports_analyst,
    expected_output="A comprehensive analysis of Manchester City's current form based on Guardiola's comments."
)

writing_task = Task(
    description='Write a sports article about Manchester City\'s form using the analysis and Guardiola\'s comments.',
    agent=journalist,
    context=[analysis_task],
    expected_output="An engaging sports article about Manchester City's current form and Guardiola's perspective."
)

# Create crew with memory
crew = Crew(
    agents=[sports_analyst, journalist],
    tasks=[analysis_task, writing_task],
    process=Process.sequential,
    memory=True,
    short_term_memory=memory,  # Explicitly pass our memory implementation
    verbose=True
)

# Run the crew
result = crew.kickoff()

print("\n" + "="*60)
print("CREWAI EXECUTION RESULT")
print("="*60)
print(result)
print("="*60)
print("\n✅ CrewAI agents successfully used Couchbase GSI memory storage!")
print("Memory will persist across sessions for continued learning.")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2fe9f129-c9c2-439d-bca8-d99540c02325                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Ensure the analysis includes specific examples or statistics to back up claims about the team's              │
│  performance.                                                                                                   │
│  - Incorporate a broader range of perspectives, including potential weaknesses or areas for improvement.        │
│  - Refer to recent match results or key player performances to provide evidence for the analysis.               │
│  - Consider including quotes from players or analysts for a more diverse viewpoint.                             │
│  - Discuss how the team's current form might impact their future matches or season goa...                       │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 1139.38ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sports Analyst                                                                                          │
│                                                                                                                 │
│  Task: Analyze Manchester City's recent performance based on Pep Guardiola's comments: "The team is playing     │
│  well, we are in a good moment. The way we are training, the way we are playing - I am really pleased."         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sports Journalist                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Manchester City's Ascension: Guardiola's Tactical Mastery and Team's Stellar Form**                          │
│                                                                                                                 │
│  As the Premier League season surges forward, Manchester City finds itself riding a wave of formidable form.    │
│  Under the astute guidance of Pep Guardiola, the team has not only demonstrated an impressive series of         │
│  performances but has also echoed the sentiments of their manager, who recently praised their current state,    │
│  stating, "The team is playing well, we are in a good moment. The way we are training, the way we are playing   │
│  - I am really pleased."                                                                                        │
│                                                                                                                 │
│  Guardiola’s comments are not mere platitudes but are supported by the tangible success on the field. In their  │
│  recent fixtures, Manchester City has exhibited consistent superiority, notching victories against some of the  │
│  league's most competitive sides. This string of positive results is a testament to their tactical brilliance   │
│  and squad depth, elements that Guardiola has meticulously cultivated.                                          │
│                                                                                                                 │
│  One cannot discuss City's current form without highlighting their attacking prowess. The synergy between       │
│  Erling Haaland and Kevin De Bruyne has been nothing short of electrifying. Haaland, continuing his             │
│  goal-scoring exploits, has been a relentless force, while De Bruyne's vision and creativity in midfield have   │
│  been instrumental in orchestrating City's offensive plays. Their performances underscore the attacking         │
│  fluidity that has become synonymous with Guardiola's teams.                                                    │
│                                                                                                                 │
│  Statistics further reinforce Manchester City's dominance. The team has consistently maintained a possession    │
│  rate surpassing 60%, showcasing their ability to control and dictate the tempo of matches. Their passing       │
│  accuracy, often hovering around the 90% mark, underlines their disciplined approach and technical excellence.  │
│  Defensively, Ruben Dias and John Stones have forged a formidable partnership, contributing to a commendably    │
│  low goals-conceded tally, a critical aspect of their recent success.                                           │
│                                                                                                                 │
│  However, even in the midst of this success, Guardiola acknowledges areas for refinement. While City's defense  │
│  stands robust, vulnerabilities during set-pieces have been observed, presenting an opportunity for             │
│  improvement. Additionally, encounters with teams employing deep defensive blocks have occasionally posed       │
│  challenges, suggesting that further tactical adjustments could be beneficial.                                  │
│                                                        

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 75fc6f42-bebf-4ca2-9ac4-f1bab08a5c32                                                                     │
│  Agent: Sports Journalist                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2fe9f129-c9c2-439d-bca8-d99540c02325                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Manchester City's Ascension: Guardiola's Tactical Mastery and Team's Stellar Form**            │
│                                                                                                                 │
│  As the Premier League season surges forward, Manchester City finds itself riding a wave of formidable form.    │
│  Under the astute guidance of Pep Guardiola, the team has not only demonstrated an impressive series of         │
│  performances but has also echoed the sentiments of their manager, who recently praised their current state,    │
│  stating, "The team is playing well, we are in a good moment. The way we are training, the way we are playing   │
│  - I am really pleased."                                                                                        │
│                                                                                                                 │
│  Guardiola’s comments are not mere platitudes but are supported by the tangible success on the field. In their  │
│  recent fixtures, Manchester City has exhibited consistent superiority, notching victories against some of the  │
│  league's most competitive sides. This string of positive results is a testament to their tactical brilliance   │
│  and squad depth, elements that Guardiola has meticulously cultivated.                                          │
│                                                                                                                 │
│  One cannot discuss City's current form without highlighting their attacking prowess. The synergy between       │
│  Erling Haaland and Kevin De Bruyne has been nothing short of electrifying. Haaland, continuing his             │
│  goal-scoring exploits, has been a relentless force, while De Bruyne's vision and creativity in midfield have   │
│  been instrumental in orchestrating City's offensive plays. Their performances underscore the attacking         │
│  fluidity that has become synonymous with Guardiola's teams.                                                    │
│                                                                                                                 │
│  Statistics further reinforce Manchester City's dominance. The team has consistently maintained a possession    │
│  rate surpassing 60%, showcasing their ability to control and dictate the tempo of matches. Their passing       │
│  accuracy, often hovering around the 90% mark, underlines their disciplined approach and technical excellence.  │
│  Defensively, Ruben Dias and John Stones have forged a formidable partnership, contributing to a commendably    │
│  low goals-conceded tally, a critical aspect of their recent success.                                           │
│                                                                                                                 │
│  However, even in the midst of this success, Guardiola acknowledges areas for refinement. While City's defense  │
│  stands robust, vulnerabilities during set-pieces have been observed, presenting an opportunity for             │
│  improvement. Additionally, encounters with teams employing deep defensive blocks have occasionally posed       │
│  challenges, suggesting that further tactical adjustme


CREWAI EXECUTION RESULT
**Manchester City's Ascension: Guardiola's Tactical Mastery and Team's Stellar Form**

As the Premier League season surges forward, Manchester City finds itself riding a wave of formidable form. Under the astute guidance of Pep Guardiola, the team has not only demonstrated an impressive series of performances but has also echoed the sentiments of their manager, who recently praised their current state, stating, "The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased."

Guardiola’s comments are not mere platitudes but are supported by the tangible success on the field. In their recent fixtures, Manchester City has exhibited consistent superiority, notching victories against some of the league's most competitive sides. This string of positive results is a testament to their tactical brilliance and squad depth, elements that Guardiola has meticulously cultivated.

One cannot discuss City's current f

## Memory Retention Testing

### Verify Memory Storage and Retrieval

Test that our agents successfully stored memories and can retrieve them using semantic search.

In [11]:
# Wait for memories to be stored
time.sleep(2)

# List all documents in the collection
try:
    # Query to fetch all documents of this memory type
    query_str = f"SELECT META().id, * FROM `{storage.bucket_name}`.`{storage.scope_name}`.`{storage.collection_name}` WHERE memory_type = $type"
    query_result = storage.cluster.query(query_str, type=storage.type)
    
    print(f"\nAll memory entries in Couchbase:")
    print("-" * 80)
    for i, row in enumerate(query_result, 1):
        doc_id = row.get('id')
        memory_id = row.get(storage.collection_name, {}).get('memory_id', 'unknown')
        content = row.get(storage.collection_name, {}).get('text', '')[:100] + "..."  # Truncate for readability
        
        print(f"Entry {i}: {memory_id}")
        print(f"Content: {content}")
        print("-" * 80)
except Exception as e:
    print(f"Failed to list memory entries: {str(e)}")

# Test memory retention
memory_query = "What is Manchester City's current form according to Guardiola?"
memory_results = storage.search(
    query=memory_query,
    limit=5,  # Increased to see more results
    score_threshold=0.0  # Lower threshold to see all results
)

print("\nMemory Search Results:")
print("-" * 80)
for result in memory_results:
    print(f"Context: {result['context']}")
    print(f"Distance: {result['distance']} (lower = more similar)")
    print("-" * 80)

# Try a more specific query to find agent interactions
interaction_query = "Manchester City playing style analysis tactical"
interaction_results = storage.search(
    query=interaction_query,
    limit=3,
    score_threshold=0.0
)

print("\nAgent Interaction Memory Results:")
print("-" * 80)
if interaction_results:
    for result in interaction_results:
        print(f"Context: {result['context'][:200]}...")  # Limit output size
        print(f"Distance: {result['distance']} (lower = more similar)")
        print("-" * 80)
else:
    print("No interaction memories found. This is normal if agents haven't completed tasks yet.")
    print("-" * 80)


All memory entries in Couchbase:
--------------------------------------------------------------------------------

Memory Search Results:
--------------------------------------------------------------------------------
Context: Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased.'
Distance: 0.2853742345382412 (lower = more similar)
--------------------------------------------------------------------------------
Context: Manchester City's recent performance, as described by manager Pep Guardiola, highlights a period of positive form for the team. Guardiola's comments, "The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased," suggest that the team is currently displaying a high level of performance both in training and during matches. 

To analyze this statement comprehensively, we can look at

## Conclusion

You've successfully implemented a custom memory backend for CrewAI agents using Couchbase GSI vector search! This system provides: